In [7]:
!pip install tensorflow opencv-python matplotlib

  Using cached opencv_python-4.8.0.74-cp37-abi3-win_amd64.whl (38.1 MB)


In [3]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer , Conv2D , Dense , MaxPooling2D , Input , Flatten
import tensorflow as tf

In [5]:
# avoid out of mem errors
gpus  = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu,True)
        

In [6]:
POS_PATH = os.path.join('data' , 'positive')
NEG_PATH = os.path.join('data' , 'negative')
ANC_PATH = os.path.join('data' , 'anchor')

In [6]:
os.mkdir(POS_PATH)
os.mkdir(NEG_PATH)
os.mkdir(ANC_PATH)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data\\positive'

In [10]:
!tar -xf lfw.tgz

In [26]:
 for directory in os.listdir('lfw'):
        for file in os.listdir(os.path.join('lfw',directory)):
            EX_PATH = os.path.join('lfw' , directory , file)
            NEW_PATH = os.path.join(NEG_PATH , file)
            os.replace(EX_PATH , NEW_PATH)

In [7]:
import uuid

In [8]:
cap  = cv2.VideoCapture(0)
while cap.isOpened():
    ret , frame = cap.read()
    
    # 250*250
    frame = frame[120:120+250,200:200+250,:]
    
    #anchor collection
    if cv2.waitKey(1) & 0XFF == ord('a'):
        #create unique file path
        imgname = os.path.join(ANC_PATH , '{}.jpg'.format(uuid.uuid1()))
        #write out anchor image
        cv2.imwrite(imgname,frame)
        
    #pos collection
    if cv2.waitKey(1) & 0XFF == ord('p'):
        #create unique file path
        imgname = os.path.join(POS_PATH , '{}.jpg'.format(uuid.uuid1()))
        #write out anchor image
        cv2.imwrite(imgname,frame)
        
    
    
    cv2.imshow('Image collection' , frame)
    
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

#release the webcam
cap.release()
cv2.destroyAllWindows()

In [9]:
anchor = tf.data.Dataset.list_files(ANC_PATH + '\*.jpg').take(150)
positive = tf.data.Dataset.list_files(POS_PATH + '\*.jpg').take(150)
negative = tf.data.Dataset.list_files(NEG_PATH + '\*.jpg').take(150)

In [10]:
dir_test = anchor.as_numpy_iterator()

In [17]:
dir_test.next()

b'data\\anchor\\e30aa3db-3488-11ee-9086-d8f88368ebcd.jpg'

In [18]:
def preprocess(file_path):
    
    #read in image from file path
    byte_img = tf.io.read_file(file_path)
    #load in the image
    img = tf.io.decode_jpeg(byte_img)
    
    #resizing to 100*100
    img = tf.image.resize(img,(100,100))
    #scale between 0 - 1
    img = img /255.0
    return img

In [19]:
img = preprocess('data\\anchor\\e30aa3db-3488-11ee-9086-d8f88368ebcd.jpg')

In [20]:
img.numpy().min()

0.10955882

In [23]:
#label the dataset
positives = tf.data.Dataset.zip((anchor,positive,tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor,negative,tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [24]:
data

<_ConcatenateDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>

In [25]:
smaples = data.as_numpy_iterator()

In [27]:
smaples.next()

(b'data\\anchor\\ef0176b8-3488-11ee-b3dc-d8f88368ebcd.jpg',
 b'data\\positive\\03682c8d-3489-11ee-8f39-d8f88368ebcd.jpg',
 1.0)

In [28]:
def preprocess_twin(input_img , validation_img,label):
    return(preprocess(input_img) , preprocess(validation_img) , label)

In [29]:
#build dataloader pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size = 1024)

In [31]:
smaples = data.as_numpy_iterator()

In [38]:
smaples.next()[2]

0.0

In [39]:
# training partition
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [41]:
smaples = train_data.as_numpy_iterator()

In [47]:
len(smaples.next()[2])

16

test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))

In [51]:
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [52]:
test_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(None, 100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [54]:
inp = Input(shape=(105,105,3) , name = 'input_image')
print(inp)

KerasTensor(type_spec=TensorSpec(shape=(None, 105, 105, 3), dtype=tf.float32, name='input_image'), name='input_image', description="created by layer 'input_image'")


In [55]:
c1 = Conv2D(64,(10,10) , activation='relu')(inp)
print(c1)

KerasTensor(type_spec=TensorSpec(shape=(None, 96, 96, 64), dtype=tf.float32, name=None), name='conv2d/Relu:0', description="created by layer 'conv2d'")


In [56]:
m1 = MaxPooling2D(64,(2,2),padding='same')(c1)
print(m1)

KerasTensor(type_spec=TensorSpec(shape=(None, 48, 48, 64), dtype=tf.float32, name=None), name='max_pooling2d/MaxPool:0', description="created by layer 'max_pooling2d'")


In [59]:
c2 = Conv2D(128,(7,7) , activation='relu')(m1)
m2 = MaxPooling2D(64,(2,2),padding='same')(c2)
c3 = Conv2D(128,(4,4) , activation='relu')(m2)
m3 = MaxPooling2D(64,(2,2),padding='same')(c3)    
c4 = Conv2D(128,(4,4) , activation='relu')(m3)
f1 = Flatten()(c4)
d1 = Dense(4096,activation='sigmoid')(f1)
print(d1)

KerasTensor(type_spec=TensorSpec(shape=(None, 4096), dtype=tf.float32, name=None), name='dense/Sigmoid:0', description="created by layer 'dense'")


In [60]:
mod  = Model(inputs = [inp], outputs =[d1] ,name ='embedding')

In [62]:
mod.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 105, 105, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 96, 96, 64)        19264     
                                                                 
 max_pooling2d (MaxPooling2  (None, 48, 48, 64)        0         
 D)                                                              
                                                                 
 conv2d_4 (Conv2D)           (None, 42, 42, 128)       401536    
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 21, 21, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 18, 18, 128)       26

In [80]:
def make_embedding():
    inp = Input(shape=(100,100,3) , name ='input_image')
    #first block
    c1 = Conv2D(64,(10,10) , activation='relu')(inp)
    m1 = MaxPooling2D(64,(2,2),padding='same')(c1)
    
    #second block
    c2 = Conv2D(128,(7,7) , activation='relu')(m1)
    m2 = MaxPooling2D(64,(2,2),padding='same')(c2)
    
    #3rd block
    c3 = Conv2D(128,(4,4) , activation='relu')(m2)
    m3 = MaxPooling2D(64,(2,2),padding='same')(c3)
    
    #final embedding block
    
    c4 = Conv2D(128,(4,4) , activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096,activation='sigmoid')(f1)
    
    return Model(inputs = [inp], outputs =[d1] ,name ='embedding')

In [81]:
embedding = make_embedding()

In [82]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 100, 100, 3)]     0         
                                                                 
 conv2d_15 (Conv2D)          (None, 91, 91, 64)        19264     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 46, 46, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_16 (Conv2D)          (None, 40, 40, 128)       401536    
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 20, 20, 128)       0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (None, 17, 17, 128)       26

In [88]:
#siamese L1 distance class
class L1Dist(Layer):
    def __init__(self,**kwargs):
        super().__init__()
        
        
    def call(self,input_embedding,validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)
    

In [89]:
l1  = L1Dist()

In [90]:
l1

In [91]:
def make_siamese_model():
    
    #Anchor image input in network
    input_image = Input(name='input_img' , shape=(100,100,3))
    
    #validation image in network
    validation_image = Input(name='validation_img' ,shape=(100,100,3))
    
    # combine siame  distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image) , embedding(validation_image))
    
    #classification layer
    classifier = Dense(1,activation='sigmoid')(distances)
    
    return Model(inputs=[input_image,validation_image],outputs=classifier , name='SiameseNetwork')

In [94]:
siamese_model = make_siamese_model()

In [96]:
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_img (InputLayer)      [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 validation_img (InputLayer  [(None, 100, 100, 3)]        0         []                            
 )                                                                                                
                                                                                                  
 embedding (Functional)      (None, 4096)                 1982380   ['input_img[0][0]',           
                                                          8          'validation_img[0][0]']      
                                                                                     